In [87]:
import math
import random
import gym
import pandas as pd

def createDictionary(numberOfXaxisBucket,numberOfSpeedBuckets,numberOfYaxisBuckets, numberOfArmSpeedBuckets):
    temp = gym.make('CartPole-v1')
    XPositionLowerBound = -2.4
    lowerXspeedBound = -0.5
    YPositionLowerBound = temp.observation_space.low[2]
    lowerYSpeedBound = -50
    XPositionCounter = math.fabs(2 * XPositionLowerBound / numberOfXaxisBucket)
    lowerXspeedCounter = math.fabs(2*lowerXspeedBound/numberOfSpeedBuckets)
    YPositionCounter = math.fabs(2*YPositionLowerBound/ numberOfYaxisBuckets)
    lowerYSpeedCounter = math.fabs(2*lowerYSpeedBound/numberOfArmSpeedBuckets)
    XaxisBuckets = []
    SpeedBuckets = []
    YaxisBuckets = []
    ArmSpeedBuckets = []
    XaxisBucketsRes = {}
    SpeedBucketsRes = {}
    YaxisBucketsRes = {}
    ArmSpeedBucketsRes = {}
    for x1 in range(numberOfXaxisBucket+1):
        XPosition = XPositionLowerBound + x1 * XPositionCounter
        XaxisBuckets.append(XPosition)
    for xs in range(numberOfSpeedBuckets+1):
        speed = lowerXspeedBound + xs * lowerXspeedCounter
        SpeedBuckets.append(speed)
    for y1 in range(numberOfYaxisBuckets+1):
        YPosition = YPositionLowerBound + y1 * YPositionCounter
        YaxisBuckets.append(YPosition)
    for sb in range(numberOfArmSpeedBuckets+1):
        lowerYSpeed = lowerYSpeedBound + sb * lowerYSpeedCounter
        ArmSpeedBuckets.append(lowerYSpeed)

    for i in range(len(XaxisBuckets) -1):
        XaxisBucketsRes[XaxisBuckets[i],XaxisBuckets[i+1]] = i + 1
    for i in range(len(SpeedBuckets) -1):
        SpeedBucketsRes[SpeedBuckets[i],SpeedBuckets[i+1]] = i + 1
    for i in range(len(YaxisBuckets) -1):
        YaxisBucketsRes[YaxisBuckets[i],YaxisBuckets[i+1]] = i + 1
    for i in range(len(ArmSpeedBuckets) -1):
        ArmSpeedBucketsRes[ArmSpeedBuckets[i],ArmSpeedBuckets[i+1]] = i + 1

    return [XaxisBucketsRes, SpeedBucketsRes, YaxisBucketsRes, ArmSpeedBucketsRes]



class QLearner:
    def __init__(self):
        self.environment = gym.make('CartPole-v1')
        self.statistics = []
        self.gamma = 0.95
        self.attempt_no = 1
        self.learning_rate = 0.8
        self.experiment_rate = 0.95
        self.current_iteration = 0
        self.df = pd.DataFrame(columns=['Experiment_rate','Learning_rate','Gamma','Reward_sum'])
        self.Qmatrix = {}
        self.upper_bounds = [
            self.environment.observation_space.high[0], #-2.4 - 2.4 pozycja wagonika na osi lewo-prawo
            0.5, # prędkość wagonika
            self.environment.observation_space.high[2], # wychylenie drążka z pozycji równowagi
            math.radians(50) # prędkość kątowa
        ]
        self.lower_bounds = [
            self.environment.observation_space.low[0],
            -0.5,
            self.environment.observation_space.low[2],
            -math.radians(50)
        ]

    def learn(self, max_attempts):
        tem_rew = 0
        for _ in range(max_attempts):
            self.current_iteration += 1
            reward_sum = self.attempt()
            print(reward_sum)
        self.df.to_excel(r'C:\Users\Michał\Desktop\iet\IWUM\Zaj2\SARSA44.xlsx', sheet_name='srednia')
    def attempt(self):
        observation = self.discretise(self.environment.reset())
        done = False
        reward_sum = 0.0
        while not done:
            #self.environment.render()
            action = self.pick_action(observation)
            new_observation, reward, done, info = self.environment.step(action)
            new_observation = self.discretise(new_observation)
            self.update_knowledge(action, observation, new_observation, reward)
            observation = new_observation
            reward_sum += reward
        self.attempt_no += 1
        print(str(self.experiment_rate) + ' ' + str(self.learning_rate) + ' ' +  str(self.gamma) + '    ' + str(reward_sum))
        self.df = self.df.append({'Experiment_rate': self.experiment_rate, 'Learning_rate' : self.learning_rate, 'Gamma' : self.gamma, 'Reward_sum' : reward_sum},ignore_index=True)
        if self.experiment_rate > 0:
            self.experiment_rate -= 0.005
        if self.experiment_rate < 0:
            self.experiment_rate = 0
        if self.learning_rate >= 0.4:
            self.learning_rate -= 0.01
        if self.gamma >= 0.2:
            self.gamma -= 0.001
        return reward_sum
    
    def discretise(self, observation):
        discretiseArray = []
        for i in range(len(observation)):
            counter = 0
            for j in listOfDict[i]:
                counter += 1
                if observation[i] < j[0] and counter == 1:
                    discretiseArray.append(listOfDict[i][j])
                    break
                elif observation[i] > j[1] and counter == len(listOfDict[i]):
                    discretiseArray.append(listOfDict[i][j])
                    break
                elif observation[i] <= j[1] and observation[i] >= j[0]:
                    discretiseArray.append(listOfDict[i][j])
                    break


        return discretiseArray[0], discretiseArray[1], discretiseArray[2], discretiseArray[3]

    def pick_action(self, observation):
        if random.uniform(0, 1) > self.experiment_rate:
            if tuple([observation,0]) not in self.Qmatrix.keys():
                return self.environment.action_space.sample()
            if tuple([observation,1]) not in self.Qmatrix.keys():
                return self.environment.action_space.sample()

            if self.Qmatrix[tuple([observation,0])] > self.Qmatrix[tuple([observation,1])]:
                return 0
            else:
                return 1
        else:
            return self.environment.action_space.sample()

    def update_knowledge(self, action, observation, new_observation, reward):
        if tuple([observation,action]) not in self.Qmatrix.keys():
            self.Qmatrix[tuple([observation, action])] = 0

        if tuple([new_observation,1]) not in self.Qmatrix.keys():
            self.Qmatrix[tuple([new_observation,1])] = 0

        if tuple([new_observation,0]) not in self.Qmatrix.keys():
            self.Qmatrix[tuple([new_observation,0])] = 0

        #new_reward = self.Qmatrix[tuple([new_observation,1])] if self.Qmatrix[tuple([new_observation,1])] > self.Qmatrix[tuple([new_observation,0])] else self.Qmatrix[tuple([new_observation,0])]
        new_reward = self.Qmatrix[tuple([new_observation, self.pick_action(new_observation)])]
        self.Qmatrix[tuple([observation, action])] = (1 - self.learning_rate) * self.Qmatrix[tuple([observation,action])] + self.learning_rate * (reward + self.gamma * new_reward)



def main():
    learner = QLearner()
    learner.learn(10000)


In [88]:
listOfDict = createDictionary(6, 4, 4, 4)
main()

[2020-03-21 15:23:44,383] Making new env: CartPole-v1
[2020-03-21 15:23:44,385] Making new env: CartPole-v1


0.95 0.8 0.95    21.0
21.0
0.945 0.79 0.949    16.0
16.0
0.94 0.78 0.948    18.0
18.0
0.9349999999999999 0.77 0.947    19.0
19.0
0.9299999999999999 0.76 0.946    16.0
16.0
0.9249999999999999 0.75 0.945    23.0
23.0
0.9199999999999999 0.74 0.944    12.0
12.0
0.9149999999999999 0.73 0.943    27.0
27.0
0.9099999999999999 0.72 0.942    28.0
28.0
0.9049999999999999 0.71 0.941    11.0
11.0
0.8999999999999999 0.7 0.94    16.0
16.0
0.8949999999999999 0.69 0.939    19.0
19.0
0.8899999999999999 0.6799999999999999 0.938    16.0
16.0
0.8849999999999999 0.6699999999999999 0.9369999999999999    16.0
16.0
0.8799999999999999 0.6599999999999999 0.9359999999999999    21.0
21.0
0.8749999999999999 0.6499999999999999 0.9349999999999999    40.0
40.0
0.8699999999999999 0.6399999999999999 0.9339999999999999    77.0
77.0
0.8649999999999999 0.6299999999999999 0.9329999999999999    13.0
13.0
0.8599999999999999 0.6199999999999999 0.9319999999999999    45.0
45.0
0.8549999999999999 0.6099999999999999 0.930999999999